In [11]:
import pandas as pd 
import xgboost as xgb

In [12]:
X_train=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/X_train_N1UvY30.csv")  
Y_train=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/y_train_or6m3Ta.csv") 
X_test=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/X_test_m4HAPAP.csv") 


In [13]:
def transfo_data(X_test):
    
    X_test['flux']=X_test['flux'].apply(abs)
    X_test['bid_ask_spread']=X_test['ask']-X_test['bid']
    
    X_test_new_base=(X_test[['obs_id']]).drop_duplicates()
    X_test_new_base.reset_index(drop=True,inplace=True)
    
    #Mean + std 
    var=['bid','ask','price','bid_size','ask_size','flux','bid_ask_spread']
    
    for k in var:
        X_test_new_base['mean_'+k]=pd.DataFrame(X_test.groupby('obs_id')[k].mean())
    
    for k in var:
        X_test_new_base['std_'+k]=pd.DataFrame(X_test.groupby('obs_id')[k].std())

    #Compte
    result_action = pd.DataFrame(X_test.groupby('obs_id')['action'].value_counts().unstack(fill_value=0))
    X_test_new_base['compte_A']=result_action.loc[:,'A']
    X_test_new_base['compte_delete']=result_action.loc[:,'D']
    X_test_new_base['compte_update']=result_action.loc[:,'U']
    
    result_place = pd.DataFrame(X_test.groupby('obs_id')['venue'].value_counts().unstack(fill_value=0))
    for venue in range(6):
        X_test_new_base['compte '+str(venue)]=result_place.loc[:,venue]
    
    result_side = pd.DataFrame(X_test.groupby('obs_id')['side'].value_counts().unstack(fill_value=0))
    X_test_new_base['compte_ask']=result_side.loc[:,'A']
    X_test_new_base['compte_bid']=result_side.loc[:,'B']
    
    result_trade = pd.DataFrame(X_test.groupby('obs_id')['trade'].value_counts().unstack(fill_value=0))
    X_test_new_base['compte_trade']=result_trade.loc[:,True]
    
    #end of the transformation
    X_test_new_base=X_test_new_base.drop(columns=['obs_id'],axis=1)
    return X_test_new_base

X_test_new_base=transfo_data(X_test)
X_train_new_base=transfo_data(X_train)

In [14]:
model = xgb.XGBClassifier(objective='multi:softmax')
Y_train_new_base=pd.DataFrame(Y_train[['eqt_code_cat']])
model.fit(X_train_new_base, Y_train_new_base)

y_pred = pd.DataFrame(model.predict(X_test_new_base))
y_pred.reset_index(inplace=True)
y_pred.columns=Y_train.columns
y_pred.to_csv('export_data.csv', index=False)

In [15]:
y_pred

,obs_id,eqt_code_cat
0,0,13
1,1,3
2,2,19
3,3,13
4,4,18
...,...,...
81595,81595,4
81596,81596,22
81597,81597,19
81598,81598,15
